In [15]:
import matplotlib as mlp
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
# os._exit(00)
import sys
import time
import codecs
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import re

# 1. preprocessing data
# 2. build model
# 2.1 encoder
# 2.2 attention
# 2.3 decoder
# 3. evaluation

# hyperparameters
batch_size = 64
epochs = 20
embedding_units = 256
units = 1024

# filename = 'drive/MyDrive/en_simple.txt'
# def preprocess_sentence(s):
#   s = s.strip()
#   s = '<s> ' + s + ' </s>'
#   return s

# def parse_data(filename):
#   lines = open(filename, encoding = 'UTF-8').read().strip().split('\n')
#   sentence_pairs = [line.split('\t') for line in lines]
#   preprocessed_sentence_pairs = [(preprocess_sentence(en), preprocess_sentence(cn)) for en, cn in sentence_pairs]
#   return zip(*preprocessed_sentence_pairs)

# target, source = parse_data(filename)
# print(en_dataset[-1])
# print(cn_dataset[-1])


## preprocessing data
source_path = "drive/MyDrive/Corpus/corpus_1w/chinese_out.txt"
target_path = "drive/MyDrive/Corpus/corpus_1w/english1w.txt"

# add <S> and </S>
def MakeDataset(file_path):
    dataset = [("<S> " + line + " </S>") for line in codecs.open(file_path, 'r', 'utf-8').read().splitlines()]
    return dataset

source = MakeDataset(source_path)
target = MakeDataset(target_path)

# convert sentence to word embedding
def tokenizer(lang):
  lang_tokenizer = keras.preprocessing.text.Tokenizer(num_words = None, filters="", split=" ") # create tokenizer object
  lang_tokenizer.fit_on_texts(lang) # text need to train
  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding="post")
  return tensor, lang_tokenizer

source_tensor, source_tokenizer = tokenizer(source)
target_tensor, target_tokenizer = tokenizer(target)

# max_length sentence of input and output dataset
def max_length(tensor):
  return max(len(t) for t in tensor)

max_length_input = max_length(source_tensor)
max_length_output = max_length(target_tensor)

print(max_length_input, max_length_output)

# # divide train and test dataset
# from sklearn.model_selection import train_test_split
# input_train, input_eval, output_train, output_eval = train_test_split(source_tensor, target_tensor, test_size=0.02)
# len(input_train), len(input_eval), len(output_train), len(output_eval)


# check the word embedding sentence
def convert(example, tokenizer):
  for t in example:
    if t != 0:
      print("%d --> %s" % (t, tokenizer.index_word[t]))

convert(source_tensor[0], source_tokenizer)
print()
convert(target_tensor[0], target_tokenizer)

# prepare the dataset
def make_dataset(source_tensor, target_tensor, batch_size, epochs, shuffle):
  dataset = tf.data.Dataset.from_tensor_slices((source_tensor, target_tensor)) # divide according to the first dimension, generate dataset
  if shuffle:
    dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder = True)
  return dataset

train_dataset = make_dataset(source_tensor, target_tensor, batch_size, epochs, True)
# eval_dataset = make_dataset(input_eval, output_eval, batch_size, 1, False)
print(train_dataset)
# print(eval_dataset)

for x, y in train_dataset.take(1):
  print(x.shape)
  print(y.shape)
  print(x)
  print(y)


64 62
3 --> <s>
1072 --> 1998
56 --> 年
1 --> ,
473 --> 经过
127 --> 统一
829 --> 部署
1 --> ,
10291 --> 伊犁州
1 --> ,
45 --> 地
10292 --> 两级
1397 --> 党委
287 --> 开始
3869 --> 尝试
42 --> 以
5381 --> 宣讲团
2 --> 的
386 --> 形式
1 --> ,
591 --> 深入
774 --> 学校
1 --> ,
1461 --> 村民
10293 --> 院落
1 --> ,
7011 --> 田间
7012 --> 地头
1 --> ,
95 --> 向
4483 --> 各族群众
54 --> 进行
7013 --> 面对面
3385 --> 宣讲
5 --> .
4 --> </s>

6 --> <s>
9 --> in
922 --> 1998
2 --> ,
1 --> the
6184 --> yili
779 --> autonomous
3434 --> prefecture
245 --> cpc
57 --> committee
3 --> and
1 --> the
6184 --> yili
3434 --> prefecture
245 --> cpc
57 --> committee
76 --> made
1869 --> unified
891 --> arrangements
3 --> and
1372 --> sent
17 --> on
10 --> a
2046 --> trial
398 --> basis
435 --> several
1185 --> propaganda
2942 --> teams
903 --> deep
91 --> into
1 --> the
742 --> schools
2 --> ,
1946 --> villagers
110 --> '
7958 --> courtyards
2 --> ,
3 --> and
530 --> fields
8 --> to
333 --> carry
52 --> out
604 --> face
12 --> -
8 --> to
12 --> -
604 --> 

In [11]:
import re
filename = 'drive/MyDrive/Corpus/seq2seq_testcorpus/en_simple.txt'

def preprocess_sentence(s):
  s = s.strip()
  s = '<s> ' + s + ' </s>'
  return s


def parse_data(filename):
  lines = open(filename, encoding = 'UTF-8').read().strip().split('\n')
  sentence_pairs = [line.split('\t') for line in lines]
  preprocessed_sentence_pairs = [(preprocess_sentence(en), preprocess_sentence(cn)) for en, cn in sentence_pairs]
  return zip(*preprocessed_sentence_pairs)

en_dataset, cn_dataset = parse_data(filename)
print(en_dataset[-1])
print(cn_dataset[-1])


<s> I mistook him for his brother . </s>
<s> 我 把 他 误认为 是 他 的 兄弟 。 </s>


In [16]:
## encoder
input_vocab_size = len(source_tokenizer.word_index) + 1
output_vocab_size = len(target_tokenizer.word_index) + 1

class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_units, encoding_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.encoding_units = encoding_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_units)
        self.gru = keras.layers.GRU(self.encoding_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state
  
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.encoding_units))

encoder = Encoder(input_vocab_size, embedding_units, units, batch_size)

sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(x, sample_hidden)
print(sample_output.shape, sample_hidden.shape)

(64, 64, 1024) (64, 1024)


In [17]:
## attention
class BahdanauAttention(keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = keras.layers.Dense(units)
    self.W2 = keras.layers.Dense(units)
    self.V = keras.layers.Dense(1)

  def call(self, decoder_hidden, encoder_outputs):
    # decoder_hidden.shape: (batch_size, units)
    # encoder_outputs.shape: (batch_size, length, units)
    decoder_hidden_with_time_axis = tf.expand_dims(decoder_hidden, 1)

    # before V: (batch_size, length, units)
    # after V: (batch_size, length, 1)
    score = self.V(tf.nn.tanh(self.W1(encoder_outputs) + self.W2(decoder_hidden_with_time_axis)))

    # shape: (batch_size, length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)
    
    # shape: (batch_size, length, units)
    context_vector = attention_weights * encoder_outputs

    # shape: (batch_size, units)
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

attention_model = BahdanauAttention(units = 10)
attention_results, attention_weights = attention_model(sample_hidden, sample_output)
print(attention_results.shape, attention_weights.shape)


(64, 1024) (64, 64, 1)


In [18]:
## decoder
class Decoder(keras.Model):
  def __init__(self, vocab_size, embedding_units, decoding_units, batch_size):
    super(Decoder, self).__init__()
    self.batch_size = batch_size
    self.decoding_units = decoding_units
    self.embedding = keras.layers.Embedding(vocab_size, embedding_units)
    self.gru = keras.layers.GRU(self.decoding_units, return_sequences= True, return_state= True, recurrent_initializer= "glorot_uniform")
    self.fc = keras.layers.Dense(vocab_size)
    self.attention = BahdanauAttention(self.decoding_units)
  
  def call(self, x, hidden, encoding_outputs):
    # context_vector.shape: (batch_size, units)
    context_vector, attention_weights = self.attention(hidden, encoding_outputs)

    # before embedding: x.shape: (batch_size, 1)
    # after embedding: x.shape: (batch_size, 1, embedding_units)
    x = self.embedding(x)

    combined_x = tf.concat([tf.expand_dims(context_vector, 1), x], axis = -1)

    # output.shape: [batch_size, 1, decoding_units]
    # state.shape: [batch_size, decoding_units]
    output, state = self.gru(combined_x)
    
    # output.shape: [batch_size, decoding_units]
    output = tf.reshape(output, (-1, output.shape[2]))
    
    #output.shape: [batch_size, vocab_size]
    output = self.fc(output)

    return output, state, attention_weights

decoder = Decoder(output_vocab_size, embedding_units, units, batch_size)
outputs = decoder(tf.random.uniform((batch_size, 1)), sample_hidden, sample_output)

decoder_output, decoder_hidden, decoder_aw = outputs
print(decoder_output.shape, decoder_hidden.shape, decoder_aw.shape)


(64, 12714) (64, 1024) (64, 64, 1)


In [19]:
optimizer = keras.optimizers.Adam()

loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits= True, reduction= "none")

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)
  mask = tf.cast(mask, dtype = loss_.dtype)
  loss_ *= mask
  return tf.reduce_mean(loss_)

@tf.function
def train_step(inp, targ, encoding_hidden):
  loss = 0
  with tf.GradientTape() as type:
    encoding_outputs, encoding_hidden = encoder(inp, encoding_hidden)
    decoding_hidden = encoding_hidden

    for t in range(0, targ.shape[1] - 1):
      decoding_input = tf.expand_dims(targ[:, t], 1)
      predictions, decoding_hidden, _ = decoder(decoding_input, decoding_hidden, encoding_outputs)
      loss += loss_function(targ[:, t+1], predictions)

    batch_loss = loss / int(targ.shape[0])
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = type.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss

steps_per_epoch = len(source_tensor) // batch_size

# checkpoint
checkpoint_dir = 'drive/Shareddrives/ruochen.katherina@gmail.com/training_checkpoints_smalldataset'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, attention_model=attention_model, decoder=decoder)

if not os.path.exists(checkpoint_dir):
  os.makedirs(checkpoint_dir)

for epoch in range(epochs):
  start = time.time()

  encoding_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, encoding_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))

# saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 5.2237
Epoch 1 Batch 100 Loss 3.2164
Epoch 2 Batch 0 Loss 3.1056
Epoch 2 Batch 100 Loss 2.7824
Epoch 2 Loss 3.0500
Time taken for 1 epoch 285.6455364227295 sec

Epoch 3 Batch 0 Loss 2.7138
Epoch 3 Batch 100 Loss 2.6548
Epoch 4 Batch 0 Loss 2.6669
Epoch 4 Batch 100 Loss 2.4393
Epoch 4 Loss 2.6526
Time taken for 1 epoch 287.8672273159027 sec

Epoch 5 Batch 0 Loss 2.3428
Epoch 5 Batch 100 Loss 2.3717
Epoch 6 Batch 0 Loss 2.4407
Epoch 6 Batch 100 Loss 2.3863
Epoch 6 Loss 2.3294
Time taken for 1 epoch 287.5759482383728 sec

Epoch 7 Batch 0 Loss 2.0804
Epoch 7 Batch 100 Loss 2.2540
Epoch 8 Batch 0 Loss 1.9964
Epoch 8 Batch 100 Loss 2.2121
Epoch 8 Loss 2.0796
Time taken for 1 epoch 287.9630467891693 sec

Epoch 9 Batch 0 Loss 2.0563
Epoch 9 Batch 100 Loss 2.0230
Epoch 10 Batch 0 Loss 1.7652
Epoch 10 Batch 100 Loss 1.8072
Epoch 10 Loss 1.8427
Time taken for 1 epoch 288.35400795936584 sec

Epoch 11 Batch 0 Loss 1.7702
Epoch 11 Batch 100 Loss 1.7845
Epoch 12 Batch 0 Loss 1.56

In [ ]:
# evaluation model (small dataset)
# load model
from nltk.translate.bleu_score import corpus_bleu

encoder = Encoder(input_vocab_size, embedding_units, units, 1)
attention_model = BahdanauAttention(units = 10)
decoder = Decoder(output_vocab_size, embedding_units, units, 1)
optimizer = keras.optimizers.Adam()
checkpoint_dir = 'drive/Shareddrives/ruochen.katherina@gmail.com/training_checkpoints_smalldataset'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, attention_model=attention_model, decoder=decoder)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

test_filepath = 'drive/MyDrive/test_simple.txt'
en_dataset_target, cn_dataset_source = parse_data(test_filepath)

def evaluation(input_sentence):
  input = [source_tokenizer.word_index[token] for token in input_sentence.split(' ')]

  input = keras.preprocessing.sequence.pad_sequences([input], maxlen= max_length_input, padding='post')
  input = tf.convert_to_tensor(input)
  # print(input.shape)
  result = ''
  encoding_hidden = encoder.initialize_hidden_state()
  encoding_output, encoding_hidden = encoder(input, encoding_hidden)
  decoding_hidden = encoding_hidden
  
  # decoding_input.shape: (1,1) --> (batch_size, length)
  decoding_input = tf.expand_dims([target_tokenizer.word_index['<s>']], 0)
    
  for t in range(max_length_output):
    predictions, decoding_hidden, attention_weights = decoder(decoding_input, decoding_hidden, encoding_output)
  
    # predictions.shape: (batch_size, vocab_size)
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += target_tokenizer.index_word[predicted_id] + ' '
  
    if target_tokenizer.index_word[predicted_id] == '</s>':
      # print(result.strip(' </s>'))
      return result.strip(' </s>')
    
    decoding_input = tf.expand_dims([predicted_id], 0)
  # print(result.strip(' </s>'))
  return result.strip(' </s>')

predictions = []
targets = []

for line in cn_dataset_source:
  result = evaluation(line)
  result = '<s> ' + result + ' </s>' 
  result = result.split() 
  predictions.append(result)

for line in en_dataset_target:
  line = line.lower().split()
  targets.append(line)

print(predictions)
print(targets)
## Calculate bleu score
score = corpus_bleu(targets, predictions)
print("Bleu Score = " + str(100*score))


[['<s>', '\ufeffhi', '.', '</s>'], ['<s>', 'hello', '!', '</s>'], ['<s>', 'run', '.', '</s>'], ['<s>', 'wait', '!', '</s>'], ['<s>', 'hello', '!', '</s>'], ['<s>', 'i', 'try', '.', '</s>'], ['<s>', 'i', 'won', '!', '</s>'], ['<s>', 'oh', 'no', '!', '</s>'], ['<s>', 'cheers', '!', '</s>'], ['<s>', 'got', 'it', '?', '</s>'], ['<s>', 'he', 'ran', '.', '</s>'], ['<s>', 'hop', 'in', '.', '</s>'], ['<s>', 'i', 'lost', '.', '</s>'], ['<s>', 'i', 'quit', '.', '</s>'], ['<s>', "i'm", 'ok', '.', '</s>'], ['<s>', 'listen', '.', '</s>'], ['<s>', 'it', "can't", 'be', '!', '</s>'], ['<s>', 'no', 'way', '!', '</s>'], ['<s>', 'really', '?', '</s>'], ['<s>', 'try', 'some', '.', '</s>'], ['<s>', 'we', 'try', '.', '</s>'], ['<s>', 'why', 'me', '?', '</s>'], ['<s>', 'ask', 'tom', '.', '</s>'], ['<s>', 'be', 'calm', '.', '</s>'], ['<s>', 'be', 'fair', '.', '</s>'], ['<s>', 'be', 'kind', '.', '</s>'], ['<s>', 'be', 'nice', '.', '</s>'], ['<s>', 'call', 'me', '.', '</s>'], ['<s>', 'call', 'us', '.', '</s>'],

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [21]:
# evaluation model
# load model
import jieba
from nltk.translate.bleu_score import corpus_bleu

encoder = Encoder(input_vocab_size, embedding_units, units, 1)
attention_model = BahdanauAttention(units = 10)
decoder = Decoder(output_vocab_size, embedding_units, units, 1)
optimizer = keras.optimizers.Adam()
checkpoint_dir = 'drive/Shareddrives/ruochen.katherina@gmail.com/training_checkpoints_smalldataset'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, attention_model=attention_model, decoder=decoder)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

test_input_filepath = 'drive/MyDrive/Corpus/corpus_1w/test_chinese.txt'
test_output_filepath = 'drive/MyDrive/Corpus/corpus_1w/test_english.txt'

def seg_depart(input_sentence):
  # Chinese word segmentation for each line in the document
  sentence_depart = jieba.cut(input_sentence.strip())
  outstr = ''
  for word in sentence_depart:
      if word != '\n':
          outstr += word
          outstr += " "
  return outstr

def evaluation(input_sentence):
  # attention_matrix = np.zeros((max_length_output, max_length_input))
  input_sentence = seg_depart(input_sentence)
  input_sentence = "<s> " + input_sentence + "</s>"
  
  input = [source_tokenizer.word_index[token] for token in input_sentence.split(' ')]

  input = keras.preprocessing.sequence.pad_sequences([input], maxlen= max_length_input, padding='post')
  input = tf.convert_to_tensor(input)
  # print(input.shape)
  result = ''
  encoding_hidden = encoder.initialize_hidden_state()
  encoding_output, encoding_hidden = encoder(input, encoding_hidden)
  decoding_hidden = encoding_hidden
  
  # decoding_input.shape: (1,1) --> (batch_size, length)
  decoding_input = tf.expand_dims([target_tokenizer.word_index['<s>']], 0)
    
  for t in range(max_length_output):
    predictions, decoding_hidden, attention_weights = decoder(decoding_input, decoding_hidden, encoding_output)
  
    # predictions.shape: (batch_size, vocab_size)
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += target_tokenizer.index_word[predicted_id] + ' '
  
    if target_tokenizer.index_word[predicted_id] == '</s>':
      # print(result.strip(' </s>'))
      return result.strip(' </s>')
    
    decoding_input = tf.expand_dims([predicted_id], 0)
  # print(result.strip(' </s>'))
  return result.strip(' </s>')

predictions = []
targets = []

for line in codecs.open(test_input_filepath, 'r', 'utf-8').read().splitlines():
  result = evaluation(line)
  result = result.split()
  predictions.append(result)

for line in codecs.open(test_output_filepath, 'r', 'utf-8').read().splitlines():
  line = line.lower().split()
  targets.append([line])

## Calculate bleu score
print(predictions)
print(targets)
score = corpus_bleu(targets, predictions)
print("Bleu Score = " + str(100*score))


[['whoever', 'pursues', '"', 'taiwan', 'independence', '"', 'will', 'not', 'come', 'to', 'no', 'good', 'end', '.'], ['qian', 'qichen', 'hoped', 'that', 'both', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'sides', 'of', 'the', 'two', 'side'], ['yi', 'il', '-', 'hwan', 'said', ':', 'dprk', 'youths', 'will', 'have', 'the', 'constant', 'development', 'of', 'friendly', ',', 'nurtured', 'by', 'the', 'dprk', 'people', "'s", 'republic', 'of', 'the', 'older', 'generation', 'of', 'the', 'older', 'generation', 'of', 'the', 'older', 'generation', 'of', 'the', 'older', 'generation', 'of', 'the', 'older', 'generation', 'of', 'the', 'older', 'generation', 'of', 'the', 'olde